In [2]:
import talib 
from talib import MA_Type
import pandas as pd
import datetime
import pandas as pd
import os
from urllib.request import urlopen
import json
import os 
import numpy as np
import tushare as ts
import datetime
from datetime import timedelta
import pyecharts.options as opts
from pyecharts.charts import Candlestick,Line
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings("ignore")

In [31]:
import tushare as ta
# 将tushare日期转换成固定格式
def str_to_date(int):
    date = str(int)
    if len(date) == 8:
        return date[0:4] + '-' + date[4:6] + '-' + date[6:8]
    else:
        return date
# 转会无 - 的str模式的date
def date_to_str(x):
    date = x
    if len(date) == 10:
        return date[0:4]  + date[5:7] + date[8:10]
    else:
        return date
token = '572c7e13b431da6b7481ca0693db0b56fc83b14bd41055e52e74b459'
pro = ts.pro_api(token)

In [18]:
# 获取正常上市的所有股票代码
def get_allstock_code(exchange='', list_status='L', fields='ts_code'):
    data = pro.stock_basic(exchange=exchange, list_status=list_status, fields=fields)
    return data

In [19]:
# 获取股票数据
def get_stock_data(code, start, end):
    stock_df = pro.daily(ts_code=code, start_date=start, end_date=end)
    stock_df['trade_date'] = stock_df['trade_date'].apply(str_to_date)
    stock_df.sort_values(['trade_date'],inplace=True)
    stock_df.reset_index(drop=True,inplace=True)
    return stock_df

In [20]:
# 股价复权
def fuquan(df):
    df['pct_chg'] = 100*(df['Close'] / df['pre_close'] - 1).round(4)
    # 股票复权
    df['fuquan'] = (1 + df['pct_chg']/100).cumprod()
    # close 是没复权的，Close是复权
    df['close'] = df['Close']
    df['Close'] = (df['fuquan'] * (df.iloc[0]['Close'] / df.iloc[0]['fuquan'])).round(2)
    
    df['Open'] = (df['Open'] / df['close'] * df['Close']).round(2)
    df['High'] = (df['High'] / df['close'] * df['Close']).round(2)
    df['Low'] = (df['Low'] / df['close'] * df['Close']).round(2)
    
    del df['close']
    
    # 分别计算当天之后 5 ，10 ，20天的涨跌幅
    df['pct_chg_5'] = (100*(df['Close'].shift(-5)/df['Close'] - 1)).round(2)
    df['pct_chg_10'] = (100*(df['Close'].shift(-10)/df['Close'] - 1)).round(2)
    df['pct_chg_20'] = (100*(df['Close'].shift(-20)/df['Close'] - 1)).round(2)
    df['pct_chg_30'] = (100*(df['Close'].shift(-30)/df['Close'] - 1)).round(2)
    
    # 计算90日bias
    df['mean_90'] = df['Close'].rolling(90).mean()
    df['bias_90'] = 100*(df['Close']/df['mean_90']-1)
    
    
    # 计算5日均线以及60日均线，当5日均线上穿60日均线，为买入信号
    df['mean_5'] = df['Close'].rolling(5).mean()
    df['mean_60'] = df['Close'].rolling(60).mean()
    df['5upcross60'] = 100*(df['mean_5']/df['mean_60']-1).round(4)
    
    # 计算MA60与MA90的偏离程度
    df['diff_MA60-MA90'] = 100*(df['mean_60']/df['mean_90'] - 1).round(4)
    
    del df['mean_5']
    del df['mean_60']
    del df['mean_90']
    # 计算成交量
    df['Vol_5'] = df['Volume'].rolling(5).mean()
    # 计算RSI
    close_p = df['Close'].values
    df["RSI"] = talib.RSI(close_p, timeperiod=14)
    # 计算当日最高点与收盘价的回撤
    df['now_high_close'] = 100*(df['Close']/df['High']-1).round(4)
    # 计算股票波动率
    df['std_60'] = round(100*(df['Close'].rolling(50).std() / df['Close'].rolling(50).mean()),4)
    
    # 30天内的最高点的回撤
    df['30_down'] = 100*(df['Close']/df['High'].shift(1).rolling(20).max()-1).round(4)
    
    # 计算一些alpha因子
#     df,extra_agg_dict = get_pos_alpha(df, extra_agg_dict={})

    # 计算一年半内最高点与当日的一个最大回撤
#     df['max_down'] = 100*(df['Close']/df['Close'].rolling(365,min_periods=90).max()-1).round(4)
    
    return df

In [89]:
def report(output_df,up_chg=0,down_chg=0):
    sum_data = len(output_df)
    sum_up = len(output_df[(output_df['trade_type']==1)])
    sum_down = len(output_df[(output_df['trade_type']==0)])
    
    buy_up_5 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_5']>up_chg))])
    buy_up_10 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_10']>up_chg))])
    buy_up_20 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_20']>up_chg))])
    buy_up_30 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_30']>up_chg))])
    
    sell_dwon_5 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_5']<down_chg))])
    sell_dwon_10 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_10']<down_chg))])
    sell_dwon_20 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_20']<down_chg))])
    sell_dwon_30 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_30']<down_chg))])
                    
    print('总数据量:{}'.format(sum_data))
    print('给出做多信号数据量:{}'.format(sum_up))      
    print('给出做空信号数据量:{}'.format(sum_down))
    
    print('在判断为上涨信号时，5天后为上涨的概率:{}'.format(buy_up_5/sum_up))
    print('在判断为上涨信号时，10天后为上涨的概率:{}'.format(buy_up_10/sum_up))
    print('在判断为上涨信号时，20天后为上涨的概率:{}'.format(buy_up_20/sum_up))
    print('在判断为上涨信号时，30天后为上涨的概率:{}'.format(buy_up_30/sum_up))
    print('\n')

In [25]:
start = '20140101'
end = '20210608'
name_dict = {'trade_date':'Date','open':'Open','high':'High',
             'low':'Low','close':'Close','vol':'Volume',
             'amount':'Value',
            }

In [21]:
all_data=pd.read_csv(r"D:\量化\每日量化选股\MACD背离选法\MACD选股进阶+筛选终版.csv.",encoding='GBK')
all_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90
0,000001.SZ,2016-11-21,1,0.65,4.16,2.35,-0.48,-0.90,57.708698,0.07,-0.42,1.1660,-0.66,0.32
1,000001.SZ,2021-04-23,1,1.35,0.02,1.13,8.24,NaN,65.337542,1.32,-1.51,5.3557,0.25,4.02
2,000002.SZ,2019-10-17,1,1.03,-2.53,-2.99,-3.54,1.64,58.428737,0.16,-1.08,1.8504,-1.64,-0.53
3,000006.SZ,2016-04-14,1,1.08,0.57,-4.58,-13.75,-12.60,50.554701,0.79,-0.80,5.7505,-6.03,-9.95
4,000006.SZ,2019-02-14,1,1.08,0.16,8.13,6.38,16.11,65.676599,1.25,-1.26,3.8168,0.32,2.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3122,603998.SH,2021-04-01,1,0.38,0.76,9.36,17.53,19.85,49.476538,0.05,-0.19,2.7518,-3.11,-5.95
3123,603999.SH,2021-03-22,1,3.14,-0.93,-0.62,5.35,-0.39,57.226326,0.21,0.00,2.7348,-2.20,-4.86
3124,605118.SH,2021-03-30,1,2.01,9.36,14.58,-0.61,-4.20,53.980823,0.05,-0.54,4.8783,-4.47,-5.92
3125,605158.SH,2021-03-01,1,1.32,-3.46,-0.09,3.90,6.06,61.151027,0.32,0.00,3.7160,-0.17,-3.56


In [35]:
all_data['Date'] = all_data['Date'].apply(date_to_str)
all_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90
0,000001.SZ,20161121,1,0.65,4.16,2.35,-0.48,-0.90,57.708698,0.07,-0.42,1.1660,-0.66,0.32
1,000001.SZ,20210423,1,1.35,0.02,1.13,8.24,NaN,65.337542,1.32,-1.51,5.3557,0.25,4.02
2,000002.SZ,20191017,1,1.03,-2.53,-2.99,-3.54,1.64,58.428737,0.16,-1.08,1.8504,-1.64,-0.53
3,000006.SZ,20160414,1,1.08,0.57,-4.58,-13.75,-12.60,50.554701,0.79,-0.80,5.7505,-6.03,-9.95
4,000006.SZ,20190214,1,1.08,0.16,8.13,6.38,16.11,65.676599,1.25,-1.26,3.8168,0.32,2.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3122,603998.SH,20210401,1,0.38,0.76,9.36,17.53,19.85,49.476538,0.05,-0.19,2.7518,-3.11,-5.95
3123,603999.SH,20210322,1,3.14,-0.93,-0.62,5.35,-0.39,57.226326,0.21,0.00,2.7348,-2.20,-4.86
3124,605118.SH,20210330,1,2.01,9.36,14.58,-0.61,-4.20,53.980823,0.05,-0.54,4.8783,-4.47,-5.92
3125,605158.SH,20210301,1,1.32,-3.46,-0.09,3.90,6.06,61.151027,0.32,0.00,3.7160,-0.17,-3.56


In [37]:
# trade_date
df = pro.daily_basic(ts_code='600621.SH',trade_date='20210601', fields='')
df

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,600621.SH,20210601,16.07,2.0964,4.4863,0.7,24.0703,24.7278,2.4421,110.8926,109.7917,0.1182,0.1182,106089.9292,106089.9292,49573.2504,1.704865e+06,1.704865e+06


In [17]:
# ts_code,trade_date,turnover_rate,volume_ratio,pe,pb
df = pro.daily_basic(ts_code='600621.SH', start_date='20130608',end_date='20210601', fields='')
df

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,600621.SH,20210601,16.07,2.0964,4.4863,0.70,24.0703,24.7278,2.4421,110.8926,109.7917,0.1182,0.1182,106089.9292,106089.9292,49573.2504,1.704865e+06,1.704865e+06
1,600621.SH,20210531,16.38,2.6362,5.6417,0.97,24.5346,25.2048,2.4892,113.0318,111.9096,0.1160,0.1160,106089.9292,106089.9292,49573.2504,1.737753e+06,1.737753e+06
2,600621.SH,20210528,16.25,2.7781,5.9453,1.15,24.3399,25.0048,2.4695,112.1347,111.0214,0.1169,0.1169,106089.9292,106089.9292,49573.2504,1.723961e+06,1.723961e+06
3,600621.SH,20210527,16.28,3.7922,8.1155,1.86,24.3848,25.0509,2.4740,112.3417,111.2264,0.1167,0.1167,106089.9292,106089.9292,49573.2504,1.727144e+06,1.727144e+06
4,600621.SH,20210526,15.70,3.0955,6.6247,1.89,23.5161,24.1584,2.3859,108.3394,107.2638,0.1210,0.1210,106089.9292,106089.9292,49573.2504,1.665612e+06,1.665612e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873,600621.SH,20130619,4.79,4.7284,6.4439,14.22,13.7146,13.8472,1.7678,2.6555,2.6161,2.0877,4.3841,52408.2351,52408.2351,38456.4829,2.510354e+05,2.510354e+05
1874,600621.SH,20130618,4.35,0.3770,0.5138,1.24,12.4548,12.5753,1.6054,2.4116,2.3758,2.2989,4.8276,52408.2351,52408.2351,38456.4829,2.279758e+05,2.279758e+05
1875,600621.SH,20130617,4.35,0.2494,0.3399,0.82,12.4548,12.5753,1.6054,2.4116,2.3758,2.2989,4.8276,52408.2351,52408.2351,38456.4829,2.279758e+05,2.279758e+05
1876,600621.SH,20130614,4.37,0.2860,0.3898,0.81,12.5120,12.6331,1.6128,2.4227,2.3867,2.2883,4.8055,52408.2351,52408.2351,38456.4829,2.290240e+05,2.290240e+05


In [24]:
ticker_list = all_data['ts_code'].unique()

In [66]:
# add_date = pd.DataFrame()
for code in ticker_list[1750:]:
    date_list = all_data[all_data['ts_code']==code]['Date'].tolist()
    for date in date_list:
        df = pro.daily_basic(ts_code=code,trade_date=date, fields='')
        add_date = add_date.append(df)
add_date

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,20161121,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.483,1.25,1.38,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07
0,000001.SZ,20210423,23.29,0.4242,0.9571,0.81,15.6237,14.8127,1.4932,2.9436,2.8714,0.936,0.936,1.940592e+06,1.940575e+06,860111.3751,4.519638e+07,4.519600e+07
0,000002.SZ,20191017,27.35,0.7002,1.2265,0.9,9.1528,8.471,2.0717,1.0384,0.9338,3.2141,3.8212,1.130214e+06,9.715170e+05,554628.8780,3.091136e+07,2.657099e+07
0,000006.SZ,20160414,8.39,1.4573,2.2163,0.82,27.1779,27.1779,2.5782,3.0995,3.0995,1.4064,1.4064,1.349995e+05,1.340266e+05,88126.7886,1.132646e+06,1.124483e+06
0,000006.SZ,20190214,5.64,1.2510,1.9560,1.4,9.4509,8.6014,1.2264,2.5728,3.042,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,603998.SH,20210401,5.25,0.2742,0.5174,0.68,28.5703,32.7371,1.961,2.0613,1.8801,2.2975,2.2975,4.294297e+04,4.294297e+04,22761.5945,2.254506e+05,2.254506e+05
0,603999.SH,20210322,5.26,0.5796,1.3747,2.1,46.8646,40.7972,1.7321,3.1175,2.7954,0.6464,0.6464,5.760000e+04,5.760000e+04,24285.4056,3.029760e+05,3.029760e+05
0,605118.SH,20210330,14.74,2.9443,2.9443,0.72,32.9829,36.4366,5.498,12.8405,13.5354,None,None,4.055000e+04,4.100000e+03,4100.0000,5.977070e+05,6.043400e+04
0,605158.SH,20210301,11.55,2.7561,2.7561,0.85,23.7531,17.948,2.401,0.8286,0.7747,None,None,3.934000e+04,9.840000e+03,9840.0000,4.543770e+05,1.136520e+05


In [58]:
len(ticker_list)

1863

In [67]:
# ticker_list[1650:1750]

In [69]:
# 删除重复的行
add_date.drop_duplicates(inplace=True)
add_date

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,20161121,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.483,1.25,1.38,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07
0,000001.SZ,20210423,23.29,0.4242,0.9571,0.81,15.6237,14.8127,1.4932,2.9436,2.8714,0.936,0.936,1.940592e+06,1.940575e+06,860111.3751,4.519638e+07,4.519600e+07
0,000002.SZ,20191017,27.35,0.7002,1.2265,0.9,9.1528,8.471,2.0717,1.0384,0.9338,3.2141,3.8212,1.130214e+06,9.715170e+05,554628.8780,3.091136e+07,2.657099e+07
0,000006.SZ,20160414,8.39,1.4573,2.2163,0.82,27.1779,27.1779,2.5782,3.0995,3.0995,1.4064,1.4064,1.349995e+05,1.340266e+05,88126.7886,1.132646e+06,1.124483e+06
0,000006.SZ,20190214,5.64,1.2510,1.9560,1.4,9.4509,8.6014,1.2264,2.5728,3.042,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,603998.SH,20210401,5.25,0.2742,0.5174,0.68,28.5703,32.7371,1.961,2.0613,1.8801,2.2975,2.2975,4.294297e+04,4.294297e+04,22761.5945,2.254506e+05,2.254506e+05
0,603999.SH,20210322,5.26,0.5796,1.3747,2.1,46.8646,40.7972,1.7321,3.1175,2.7954,0.6464,0.6464,5.760000e+04,5.760000e+04,24285.4056,3.029760e+05,3.029760e+05
0,605118.SH,20210330,14.74,2.9443,2.9443,0.72,32.9829,36.4366,5.498,12.8405,13.5354,None,None,4.055000e+04,4.100000e+03,4100.0000,5.977070e+05,6.043400e+04
0,605158.SH,20210301,11.55,2.7561,2.7561,0.85,23.7531,17.948,2.401,0.8286,0.7747,None,None,3.934000e+04,9.840000e+03,9840.0000,4.543770e+05,1.136520e+05


In [83]:
# all_features_data = pd.merge(all_data,add_date,how='left',left_on = ['ts_code','Date'],right_on = ['ts_code','trade_date'])
all_features_data = pd.DataFrame()

In [74]:
# add_date.isnull().any()

ts_code            False
trade_date         False
close              False
turnover_rate      False
turnover_rate_f    False
volume_ratio        True
pe                  True
pe_ttm              True
pb                  True
ps                 False
ps_ttm              True
dv_ratio            True
dv_ttm              True
total_share        False
float_share        False
free_share         False
total_mv           False
circ_mv            False
dtype: bool

In [76]:
yiluo_df = all_features_data[all_features_data['close'].isna()]

In [81]:
yiluo_list = yiluo_df['ts_code'].unique()
yiluo_list

array(['000677.SZ', '000678.SZ', '000679.SZ', '000680.SZ', '000681.SZ',
       '000682.SZ', '000683.SZ', '000685.SZ', '000686.SZ', '000688.SZ',
       '000690.SZ', '000691.SZ', '000692.SZ', '000695.SZ', '000697.SZ',
       '000698.SZ', '000700.SZ', '000701.SZ', '000702.SZ', '000703.SZ',
       '000708.SZ', '000709.SZ', '000711.SZ', '000712.SZ', '000713.SZ',
       '000715.SZ', '000717.SZ', '000718.SZ', '000719.SZ', '000720.SZ',
       '000721.SZ', '000722.SZ', '000723.SZ', '000725.SZ', '000726.SZ',
       '000728.SZ', '000729.SZ', '000731.SZ', '000732.SZ', '000733.SZ',
       '000735.SZ', '000736.SZ', '000737.SZ', '000738.SZ', '000739.SZ',
       '000750.SZ', '000751.SZ', '000753.SZ', '000755.SZ', '000756.SZ',
       '000757.SZ', '000759.SZ', '000761.SZ', '000762.SZ', '000766.SZ',
       '000767.SZ', '000768.SZ', '000777.SZ', '000778.SZ', '000779.SZ',
       '000788.SZ', '000791.SZ', '000793.SZ', '000797.SZ', '000799.SZ',
       '000800.SZ', '000801.SZ', '000807.SZ', '000810.SZ', '0008

In [82]:
for code in yiluo_list:
    date_list = all_data[all_data['ts_code']==code]['Date'].tolist()
    for date in date_list:
        df = pro.daily_basic(ts_code=code,trade_date=date, fields='')
        add_date = add_date.append(df)
add_date

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,20161121,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.483,1.25,1.38,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07
0,000001.SZ,20210423,23.29,0.4242,0.9571,0.81,15.6237,14.8127,1.4932,2.9436,2.8714,0.936,0.936,1.940592e+06,1.940575e+06,860111.3751,4.519638e+07,4.519600e+07
0,000002.SZ,20191017,27.35,0.7002,1.2265,0.9,9.1528,8.471,2.0717,1.0384,0.9338,3.2141,3.8212,1.130214e+06,9.715170e+05,554628.8780,3.091136e+07,2.657099e+07
0,000006.SZ,20160414,8.39,1.4573,2.2163,0.82,27.1779,27.1779,2.5782,3.0995,3.0995,1.4064,1.4064,1.349995e+05,1.340266e+05,88126.7886,1.132646e+06,1.124483e+06
0,000006.SZ,20190214,5.64,1.2510,1.9560,1.4,9.4509,8.6014,1.2264,2.5728,3.042,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,000885.SZ,20190218,7.91,1.0881,4.0165,1.42,6.513,8.7752,1.7087,0.8318,1.6868,0.1264,0.1264,4.963820e+04,4.483820e+04,12147.4948,3.926381e+05,3.546701e+05
0,000885.SZ,20200610,9.92,0.3533,1.0347,0.9,7.8895,13.0417,1.7402,2.1759,2.4644,2.3185,3.5887,4.963820e+04,4.963820e+04,16947.4948,4.924109e+05,4.924109e+05
0,000886.SZ,20161122,5.72,1.1187,1.5084,1.01,None,None,2.2099,26.6496,29.2143,None,None,9.888283e+04,9.653329e+04,71593.2002,5.656098e+05,5.521704e+05
0,000886.SZ,20210401,4.84,2.3557,3.1727,2.24,76.7442,51.8682,1.7271,27.5210,56.9767,1.0331,1.0331,9.888283e+04,9.719602e+04,72169.0628,4.785929e+05,4.704287e+05


In [84]:
all_features_data = pd.merge(all_data,add_date,how='left',left_on = ['ts_code','Date'],right_on = ['ts_code','trade_date'])
all_features_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,20161121,1,0.65,4.16,2.35,-0.48,-0.90,57.708698,0.07,-0.42,1.1660,-0.66,0.32,20161121,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.483,1.25,1.38,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07
1,000001.SZ,20210423,1,1.35,0.02,1.13,8.24,NaN,65.337542,1.32,-1.51,5.3557,0.25,4.02,20210423,23.29,0.4242,0.9571,0.81,15.6237,14.8127,1.4932,2.9436,2.8714,0.936,0.936,1.940592e+06,1.940575e+06,860111.3751,4.519638e+07,4.519600e+07
2,000002.SZ,20191017,1,1.03,-2.53,-2.99,-3.54,1.64,58.428737,0.16,-1.08,1.8504,-1.64,-0.53,20191017,27.35,0.7002,1.2265,0.9,9.1528,8.471,2.0717,1.0384,0.9338,3.2141,3.8212,1.130214e+06,9.715170e+05,554628.8780,3.091136e+07,2.657099e+07
3,000006.SZ,20160414,1,1.08,0.57,-4.58,-13.75,-12.60,50.554701,0.79,-0.80,5.7505,-6.03,-9.95,20160414,8.39,1.4573,2.2163,0.82,27.1779,27.1779,2.5782,3.0995,3.0995,1.4064,1.4064,1.349995e+05,1.340266e+05,88126.7886,1.132646e+06,1.124483e+06
4,000006.SZ,20190214,1,1.08,0.16,8.13,6.38,16.11,65.676599,1.25,-1.26,3.8168,0.32,2.11,20190214,5.64,1.2510,1.9560,1.4,9.4509,8.6014,1.2264,2.5728,3.042,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3122,603998.SH,20210401,1,0.38,0.76,9.36,17.53,19.85,49.476538,0.05,-0.19,2.7518,-3.11,-5.95,20210401,5.25,0.2742,0.5174,0.68,28.5703,32.7371,1.961,2.0613,1.8801,2.2975,2.2975,4.294297e+04,4.294297e+04,22761.5945,2.254506e+05,2.254506e+05
3123,603999.SH,20210322,1,3.14,-0.93,-0.62,5.35,-0.39,57.226326,0.21,0.00,2.7348,-2.20,-4.86,20210322,5.26,0.5796,1.3747,2.1,46.8646,40.7972,1.7321,3.1175,2.7954,0.6464,0.6464,5.760000e+04,5.760000e+04,24285.4056,3.029760e+05,3.029760e+05
3124,605118.SH,20210330,1,2.01,9.36,14.58,-0.61,-4.20,53.980823,0.05,-0.54,4.8783,-4.47,-5.92,20210330,14.74,2.9443,2.9443,0.72,32.9829,36.4366,5.498,12.8405,13.5354,None,None,4.055000e+04,4.100000e+03,4100.0000,5.977070e+05,6.043400e+04
3125,605158.SH,20210301,1,1.32,-3.46,-0.09,3.90,6.06,61.151027,0.32,0.00,3.7160,-0.17,-3.56,20210301,11.55,2.7561,2.7561,0.85,23.7531,17.948,2.401,0.8286,0.7747,None,None,3.934000e+04,9.840000e+03,9840.0000,4.543770e+05,1.136520e+05


In [85]:
all_features_data.isnull().any()

ts_code            False
Date               False
trade_type         False
pct_chg            False
pct_chg_5           True
pct_chg_10          True
pct_chg_20          True
pct_chg_30          True
RSI                False
5upcross60         False
now_high_close     False
std_60             False
30_down            False
diff_MA60-MA90     False
trade_date         False
close              False
turnover_rate      False
turnover_rate_f    False
volume_ratio        True
pe                  True
pe_ttm              True
pb                  True
ps                 False
ps_ttm              True
dv_ratio            True
dv_ttm              True
total_share        False
float_share        False
free_share         False
total_mv           False
circ_mv            False
dtype: bool

In [86]:
del all_features_data['trade_date']

In [87]:
all_features_data['Date'] = all_features_data['Date'].apply(str_to_date)
all_features_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,2016-11-21,1,0.65,4.16,2.35,-0.48,-0.90,57.708698,0.07,-0.42,1.1660,-0.66,0.32,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.483,1.25,1.38,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07
1,000001.SZ,2021-04-23,1,1.35,0.02,1.13,8.24,NaN,65.337542,1.32,-1.51,5.3557,0.25,4.02,23.29,0.4242,0.9571,0.81,15.6237,14.8127,1.4932,2.9436,2.8714,0.936,0.936,1.940592e+06,1.940575e+06,860111.3751,4.519638e+07,4.519600e+07
2,000002.SZ,2019-10-17,1,1.03,-2.53,-2.99,-3.54,1.64,58.428737,0.16,-1.08,1.8504,-1.64,-0.53,27.35,0.7002,1.2265,0.9,9.1528,8.471,2.0717,1.0384,0.9338,3.2141,3.8212,1.130214e+06,9.715170e+05,554628.8780,3.091136e+07,2.657099e+07
3,000006.SZ,2016-04-14,1,1.08,0.57,-4.58,-13.75,-12.60,50.554701,0.79,-0.80,5.7505,-6.03,-9.95,8.39,1.4573,2.2163,0.82,27.1779,27.1779,2.5782,3.0995,3.0995,1.4064,1.4064,1.349995e+05,1.340266e+05,88126.7886,1.132646e+06,1.124483e+06
4,000006.SZ,2019-02-14,1,1.08,0.16,8.13,6.38,16.11,65.676599,1.25,-1.26,3.8168,0.32,2.11,5.64,1.2510,1.9560,1.4,9.4509,8.6014,1.2264,2.5728,3.042,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3122,603998.SH,2021-04-01,1,0.38,0.76,9.36,17.53,19.85,49.476538,0.05,-0.19,2.7518,-3.11,-5.95,5.25,0.2742,0.5174,0.68,28.5703,32.7371,1.961,2.0613,1.8801,2.2975,2.2975,4.294297e+04,4.294297e+04,22761.5945,2.254506e+05,2.254506e+05
3123,603999.SH,2021-03-22,1,3.14,-0.93,-0.62,5.35,-0.39,57.226326,0.21,0.00,2.7348,-2.20,-4.86,5.26,0.5796,1.3747,2.1,46.8646,40.7972,1.7321,3.1175,2.7954,0.6464,0.6464,5.760000e+04,5.760000e+04,24285.4056,3.029760e+05,3.029760e+05
3124,605118.SH,2021-03-30,1,2.01,9.36,14.58,-0.61,-4.20,53.980823,0.05,-0.54,4.8783,-4.47,-5.92,14.74,2.9443,2.9443,0.72,32.9829,36.4366,5.498,12.8405,13.5354,None,None,4.055000e+04,4.100000e+03,4100.0000,5.977070e+05,6.043400e+04
3125,605158.SH,2021-03-01,1,1.32,-3.46,-0.09,3.90,6.06,61.151027,0.32,0.00,3.7160,-0.17,-3.56,11.55,2.7561,2.7561,0.85,23.7531,17.948,2.401,0.8286,0.7747,None,None,3.934000e+04,9.840000e+03,9840.0000,4.543770e+05,1.136520e+05


In [88]:
# all_features_data.to_csv(r"D:\量化\每日量化选股\MACD背离选法\MACD选股进阶+筛选终版+基本面指标.csv.",index=False,encoding='GBK')

In [98]:
up_chg=0
down_chg=0
report(all_up_data,up_chg,down_chg)

总数据量:2772
给出做多信号数据量:2772
给出做空信号数据量:0
在判断为上涨信号时，5天后为上涨的概率:0.48665223665223667
在判断为上涨信号时，10天后为上涨的概率:0.5180375180375181
在判断为上涨信号时，20天后为上涨的概率:0.5057720057720058
在判断为上涨信号时，30天后为上涨的概率:0.5119047619047619




In [101]:
all_up_data = all_features_data.copy()
print('5days',all_up_data['pct_chg_5'].mean())
print('10days',all_up_data['pct_chg_10'].mean())
print('20days',all_up_data['pct_chg_20'].mean())
print('30days',all_up_data['pct_chg_30'].mean())

5days 0.49368826170622265
10days 0.9896967741935493
20days 2.1181939040207514
30days 3.499607971251218


In [99]:
all_up_data = all_features_data[(all_features_data['close']>=2) & (all_features_data['turnover_rate']>=0.10) ]
all_up_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,2016-11-21,1,0.65,4.16,2.35,-0.48,-0.90,57.708698,0.07,-0.42,1.1660,-0.66,0.32,9.24,0.5811,1.1172,1.43,7.2561,6.9451,0.8899,1.6499,1.483,1.25,1.38,1.717041e+06,1.463118e+06,761028.1784,1.586546e+07,1.351921e+07
1,000001.SZ,2021-04-23,1,1.35,0.02,1.13,8.24,NaN,65.337542,1.32,-1.51,5.3557,0.25,4.02,23.29,0.4242,0.9571,0.81,15.6237,14.8127,1.4932,2.9436,2.8714,0.936,0.936,1.940592e+06,1.940575e+06,860111.3751,4.519638e+07,4.519600e+07
2,000002.SZ,2019-10-17,1,1.03,-2.53,-2.99,-3.54,1.64,58.428737,0.16,-1.08,1.8504,-1.64,-0.53,27.35,0.7002,1.2265,0.9,9.1528,8.471,2.0717,1.0384,0.9338,3.2141,3.8212,1.130214e+06,9.715170e+05,554628.8780,3.091136e+07,2.657099e+07
3,000006.SZ,2016-04-14,1,1.08,0.57,-4.58,-13.75,-12.60,50.554701,0.79,-0.80,5.7505,-6.03,-9.95,8.39,1.4573,2.2163,0.82,27.1779,27.1779,2.5782,3.0995,3.0995,1.4064,1.4064,1.349995e+05,1.340266e+05,88126.7886,1.132646e+06,1.124483e+06
4,000006.SZ,2019-02-14,1,1.08,0.16,8.13,6.38,16.11,65.676599,1.25,-1.26,3.8168,0.32,2.11,5.64,1.2510,1.9560,1.4,9.4509,8.6014,1.2264,2.5728,3.042,3.2092,3.2092,1.349995e+05,1.348308e+05,86230.9824,7.613972e+05,7.604457e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3122,603998.SH,2021-04-01,1,0.38,0.76,9.36,17.53,19.85,49.476538,0.05,-0.19,2.7518,-3.11,-5.95,5.25,0.2742,0.5174,0.68,28.5703,32.7371,1.961,2.0613,1.8801,2.2975,2.2975,4.294297e+04,4.294297e+04,22761.5945,2.254506e+05,2.254506e+05
3123,603999.SH,2021-03-22,1,3.14,-0.93,-0.62,5.35,-0.39,57.226326,0.21,0.00,2.7348,-2.20,-4.86,5.26,0.5796,1.3747,2.1,46.8646,40.7972,1.7321,3.1175,2.7954,0.6464,0.6464,5.760000e+04,5.760000e+04,24285.4056,3.029760e+05,3.029760e+05
3124,605118.SH,2021-03-30,1,2.01,9.36,14.58,-0.61,-4.20,53.980823,0.05,-0.54,4.8783,-4.47,-5.92,14.74,2.9443,2.9443,0.72,32.9829,36.4366,5.498,12.8405,13.5354,None,None,4.055000e+04,4.100000e+03,4100.0000,5.977070e+05,6.043400e+04
3125,605158.SH,2021-03-01,1,1.32,-3.46,-0.09,3.90,6.06,61.151027,0.32,0.00,3.7160,-0.17,-3.56,11.55,2.7561,2.7561,0.85,23.7531,17.948,2.401,0.8286,0.7747,None,None,3.934000e+04,9.840000e+03,9840.0000,4.543770e+05,1.136520e+05


In [102]:
all_up_data.to_csv(r"D:\量化\每日量化选股\MACD背离选法\机器学习+基本面指标.csv.",index=False,encoding='GBK')